In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import tensorflow as tf
from six.moves import cPickle
import os
#import multiprocessing

import keras
import keras.backend.tensorflow_backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense, Activation
from keras.layers import Flatten
from keras.engine.topology import Input
from keras.optimizers import Adam
from keras import regularizers
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=(22,24,1)))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation="relu", 
                 #data_format = "channels_first",
                 kernel_regularizer=regularizers.l2(0.01), 
                 #input_shape=(23,24,1),
                 padding='valid',
                 name="conv2d1"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid', 
                 #data_format = "channels_first",
                 name="maxpool2d1",
                 strides=(2,2)))
#model.add(Dropout(rate=0.1))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), activation="relu", 
                 kernel_regularizer=regularizers.l2(0.01),
                 padding='valid', 
                 #data_format = "channels_first",
                 name="conv2d2"))
#model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Flatten(
    #data_format = "channels_first", 
    name="flatten"))
#model.add(Dense(units=512, kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(rate=0.5))
#model.add(Dense(units=512, kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(units=7, activation="softmax", name="preds"))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#model.load_weights('/gdrive/My Drive/workflow/periodic/code/experiments/cnn/model.h5')
model.load_weights('model.h5')
model.summary()

W0810 13:05:36.862954 23864 deprecation.py:506] From f:\anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d1 (Conv2D)             (None, 18, 20, 16)        416       
_________________________________________________________________
maxpool2d1 (MaxPooling2D)    (None, 9, 10, 16)         0         
_________________________________________________________________
conv2d2 (Conv2D)             (None, 5, 6, 64)          25664     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
preds (Dense)                (None, 7)                 13447     
Total params: 39,527
Trainable params: 39,527
Non-trainable params: 0
_________________________________________________________________


In [3]:
classes=[1,2,4,5,6,8,13]
bins=10
#os.mkdir("blank_v2_test")

x=np.load("X_test.npy")
(l,m,n,o)=x.shape
y=np.load("y_test.npy")
p=np.load("preds_proba.npy")
pr=np.load("preds.npy")

In [4]:
diff_prob=np.zeros((l,m,n))

c={1:0,2:1,4:2,5:3,6:4,8:5,13:6}

correct_classifications=np.where(y==pr)
threshold=np.where(p[correct_classifications[0]]>=0.8)

t=threshold
w=correct_classifications
m_a=diff_prob

In [5]:
for j in range(m):
    for k in range(n):
        x=np.load("X_test.npy")
        for i in range(len(t[0])):
            x[w[0][t[0][i]],j,k,0]=0.0
        y_test=model.predict(x)
        for i in range(len(t[0])):
            m_a[w[0][t[0][i]],j,k]=y_test[w[0][t[0][i]],c[y[w[0][t[0][i]]]]]-p[w[0][t[0][i]],c[y[w[0][t[0][i]]]]]
        print(j*k)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
0
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
0
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
0
7
14
21
28
35
42
49
56
63
70
77
84
91
98
105
112
119
126
133
140
147
154
161
0
8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
0
9
18
27
36
45
54
63
72
81
90
99
108
117
126
135
144
153
162
171
180
189
198
207
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
0
11
22
33
44
55
66
77
88
99
110
121
132
143
154
165
176
187
198
209
220
231
242
253
0
12
24
36
48
60
72
84
96
108
120
132
144
156
168
180
192
204
216
228
240
252
264
276
0
13
26
39
52
65
78
91
104
117
130
14

In [18]:
clss='RRd'
#os.mkdir("blanking")
os.mkdir("blanking/"+clss+"/")
os.mkdir("blanking/"+clss+"/dmdt/")
os.mkdir("blanking/"+clss+"/blank/")

%matplotlib inline
for i in range(x.shape[0]):
  if y[i]==pr[i] and max(p[i])>=0.8 and pr[i]==6:
    plt.figure()
    plt.axis("off")
    plt.imshow(x[i][:,:,0])
    plt.savefig("blanking/"+clss+"/dmdt/"+str(i)+".png", bbox_inches='tight', transparent="True", pad_inches=0)
    plt.close()
    plt.figure()
    plt.axis("off")
    plt.imshow(m_a[i])
    plt.savefig("blanking/"+clss+"/blank/"+str(i)+".png", bbox_inches='tight', transparent="True", pad_inches=0)
    plt.close()

In [13]:
np.where(y==1)

(array([   0,    1,    2, ..., 9123, 9124, 9125], dtype=int64),)

In [8]:
%cd /content/
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git
%cd pix2pix-tensorflow

/content
Cloning into 'pix2pix-tensorflow'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 14.34 MiB/s, done.
Resolving deltas: 100% (103/103), done.
/content/pix2pix-tensorflow


In [26]:
!python pix2pix-tensorflow-master/tools/process.py \
  --input_dir "blanking/EW/dmdt/" \
  --b_dir "blanking/EW/blank/" \
  --operation combine \
  --output_dir "blanking/EW/combined/"

skipping 4482 files that already exist
processing 0 files


W0623 08:52:34.001270  2428 deprecation_wrapper.py:119] From C:\Users\Meet\Desktop\xai\pix2pix-tensorflow-master\tools\tfimage.py:22: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.

W0623 08:52:34.002269  2428 deprecation_wrapper.py:119] From C:\Users\Meet\Desktop\xai\pix2pix-tensorflow-master\tools\tfimage.py:23: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 08:52:34.124065  2428 deprecation.py:323] From F:\Anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow\python\ops\math_ops.py:3743: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0623 08:52:34.136681  2428 deprecation.py:323] From F:\Anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow\python\ops\math_ops.py:3744: setdiff1d (from tensorflow.python.ops.array_ops) 